In [2]:
import os
import zipfile
import dotenv
import subprocess
import pandas as pd
from typing import Dict, Any, List
import json
import yaml
import time
import glob
import hashlib
import shutil
import logging
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import os
import subprocess
import dotenv
import time
import glob
import hashlib
import shutil
import logging
import torch

config_train_embedding_r = "config/UrbanElementsReID_train_embedding_r.yml"
config_train_plusplus = "config/UrbanElementsReID_train_patplusplus.yml"
config_test_patplusplus = "config/UrbanElementsReID_test_patplusplus.yml"
track = "assets/models/PAT_plusplus/track.txt"
submission_file_name = "assets/models/PAT_plusplus/track_submission.csv"

competition_name = "urban-reid-challenge"
submission_message = f"PAT++, simple matching, pretrain 60 epoch, keep all keys except bottleneck and classifier"


################ Probably nothing has to be modified from now on ################
assert dotenv.load_dotenv('../../.env')
assert os.getenv('KAGGLE_USERNAME')

from kaggle.api.kaggle_api_extended import KaggleApi, SubmissionStatus
api = KaggleApi()
api.authenticate()

logging.root.setLevel(logging.INFO)

if not torch.cuda.is_available():
    logging.warning("Where is your GPU dude?")

assert os.path.exists('assets'), 'are you are in the right folder?'
assert os.getcwd().endswith('PAT'), 'are you are in the right folder?'

!nvidia-smi

Mon May 12 19:25:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-16GB           Off |   00000000:AF:00.0 Off |                    0 |
| N/A   60C    P0             46W /  250W |       4MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!python train.py --config_file {config_train_embedding_r}
#!cp assets/models/PAT_r/part_attention_vit_60.pth assets/models/PAT_r/jx_vit_base_p16_224-80ecf9dd.pth
!cp assets/models/PAT_r/part_attention_vit_1.pth assets/models/PAT_r/jx_vit_base_p16_224-80ecf9dd.pth
!ls -lah assets/models/PAT_r/

2025-05-12 19:25:55,459 PAT INFO: Saving model in the path :assets/models/PAT_r
2025-05-12 19:25:55,460 PAT INFO: Namespace(config_file='config/UrbanElementsReID_train_embedding_r.yml', opts=[], local_rank=0)
2025-05-12 19:25:55,460 PAT INFO: Loaded configuration file config/UrbanElementsReID_train_embedding_r.yml
2025-05-12 19:25:55,460 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models" 
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID',)
  ROOT_DIR: 'assets/datase

In [8]:
!python train.py --config_file {config_train_plusplus}

2025-05-12 20:41:55,407 PAT INFO: Saving model in the path :assets/models/PAT_plusplus
2025-05-12 20:41:55,407 PAT INFO: Namespace(config_file='config/UrbanElementsReID_train_patplusplus.yml', opts=[], local_rank=0)
2025-05-12 20:41:55,408 PAT INFO: Loaded configuration file config/UrbanElementsReID_train_patplusplus.yml
2025-05-12 20:41:55,408 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models/PAT_r" 
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID',)
  #ROOT_DIR: 

In [9]:
!python update.py --config_file {config_test_patplusplus} --track {track}

2025-05-12 21:31:07,472 PAT INFO: Namespace(config_file='config/UrbanElementsReID_test_patplusplus.yml', opts=[], track='assets/models/PAT_plusplus/track.txt')
2025-05-12 21:31:07,472 PAT INFO: Loaded configuration file config/UrbanElementsReID_test_patplusplus.yml
2025-05-12 21:31:07,473 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models/PAT_r" 
  METRIC_LOSS_TYPE: 'triplet'
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID_test',)
  #ROOT_DIR: ('../../data') # root 

In [10]:
submission_message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
logging.info(f'Submitting with message "{submission_message}"')

api.competition_submit(submission_file_name, submission_message, competition_name)    

INFO:root:Submitting with message "PAT++, simple matching, pretrain 60 epoch, keep all keys except bottleneck and classifier; commit_hash: 9b3d05be76bd8535482da8c6d8d63b134e3a8468"
100%|██████████| 135k/135k [00:00<00:00, 177kB/s]


{"message": "Successfully submitted to Urban Elements ReID Challenge", "ref": 44714923}

In [11]:
submissions = api.competition_submissions(competition_name)
submissions.sort(key=lambda x: x.date, reverse=True)
latest = submissions[0]
while(latest.status == SubmissionStatus.PENDING):
    submissions = api.competition_submissions(competition_name)
    submissions.sort(key=lambda x: x.date, reverse=True)
    latest = submissions[0]
    print('.')
    time.sleep(1)

print(f"Status: {latest.status}")
print(f"Public Score: {latest.public_score}")
print(f"Private Score: {latest.private_score}")

.
.
.
.
.
.
.
.
.
.
.
.
Status: SubmissionStatus.COMPLETE
Public Score: 0.13972
Private Score: 


In [ ]:
# New fucked up baseline: 0.17540
# result with 60 epochs of refinement pretraining: 0.13972

In [ ]:
len(submissions)